In [1]:
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
# Example usage
topic_name = 'output-topic'
bootstrap_servers = ["10.237.96.122:9092"]
N_PARTITIONS = 1
def delete_kafka_topic(topic_name, bootstrap_servers=bootstrap_servers):
    # Create an admin client
    admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers[0])

    # Delete the topic
    admin_client.delete_topics([topic_name])
    print(f"Topic '{topic_name}' deleted successfully.")


try:
    delete_kafka_topic(topic_name)
except Exception as e:
    print(f"Error: {e}")

# create topic foobar with N partitions
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
admin_client.create_topics([NewTopic(name=topic_name, num_partitions=N_PARTITIONS, replication_factor=1)])


# get number of partitions
admin_client.describe_topics(topic_name)

In [1]:
import time
import pandas as pd
import json
from kafka import KafkaConsumer

# Create a KafkaConsumer instance
consumer = KafkaConsumer(
    # bootstrap_servers=['localhost:9092'],
    bootstrap_servers=["10.237.96.122:9092"],
    auto_offset_reset='latest',
    # auto_offset_reset='earliest',
    # receive_buffer_bytes=160*1024,
    enable_auto_commit=False,
    # value_deserializer=lambda m: json.loads(m.decode('ascii')),
)

# Subscribe to a specific topic
TOPIC_NAME = "output-topic"
consumer.subscribe(topics=[TOPIC_NAME])
N_sample =  - 1
while True:
    msg_poll = consumer.poll(timeout_ms=0)
    for k, v in msg_poll.items():
        print(k, v)
    # break

# df_record = pd.DataFrame(lst_record)
# df_record.describe()
msg_poll = consumer.poll(timeout_ms=0)
for k, v in msg_poll.items():
    print(k, v)

TopicPartition(topic='output-topic', partition=0) [ConsumerRecord(topic='output-topic', partition=0, offset=5015, timestamp=1732085584004, timestamp_type=0, key=None, value=b'setup 1', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=7, serialized_header_size=-1), ConsumerRecord(topic='output-topic', partition=0, offset=5016, timestamp=1732085584004, timestamp_type=0, key=None, value=b'sh 1', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=4, serialized_header_size=-1), ConsumerRecord(topic='output-topic', partition=0, offset=5017, timestamp=1732085584004, timestamp_type=0, key=None, value=b'wget 1', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=6, serialized_header_size=-1), ConsumerRecord(topic='output-topic', partition=0, offset=5018, timestamp=1732085584004, timestamp_type=0, key=None, value=b'https 1', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=7, serialized_header_size=-1), C